# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "../output_data/cities.csv"
vacay_df = pd.read_csv(csv_path)
vacay_df = vacay_df.drop('Unnamed: 0', axis=1)
vacay_df.head()

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp
0,6201424,Mataura,-46.19,168.86,77.65,58,64,0.58,77.65
1,1819729,Hong Kong,22.29,114.16,61.66,77,40,11.41,63.00
2,1507390,Dikson,73.51,80.55,-5.84,95,100,5.39,-5.84
3,2075265,Busselton,-33.65,115.33,90.73,24,97,5.23,93.00
4,4030556,Rikitea,-23.12,-134.97,79.11,77,0,10.22,79.11


In [4]:
# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
gmaps.configure(api_key=g_key)
locations = vacay_df[["lat", "lng"]]
humidity = vacay_df['humidity'].astype(float)
fig = gmaps.figure()

In [5]:
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = .5)

fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacay_filtered = vacay_df.loc[(vacay_df['temp']<80) & (vacay_df['temp']>70) & (vacay_df['wind speed']<10) & (vacay_df['cloudiness']==0)]

vacay_filtered

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp
70,3471451,Arraial do Cabo,-22.97,-42.03,75.16,82,0,6.76,75.16
173,3836669,San Rafael,-34.62,-68.33,77.09,28,0,2.57,77.09
185,3867427,Mayor Pablo Lagerenza,-19.93,-60.77,78.64,41,0,4.50,78.64
364,3455756,Osvaldo Cruz,-21.80,-50.88,73.40,69,0,3.36,73.40
448,3860259,Cordoba,-31.41,-64.18,72.97,64,0,5.82,79.00
456,3520161,Sabancuy,18.97,-91.18,70.57,86,0,4.07,70.57
457,1272140,Dwarka,22.24,68.97,71.55,70,0,9.46,71.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacay_df
hotel_df['Hotel Name'] = ''
hotel_df['address'] = ''
hotel_df.head()

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp,Hotel Name,address
0,6201424,Mataura,-46.19,168.86,77.65,58,64,0.58,77.65,,
1,1819729,Hong Kong,22.29,114.16,61.66,77,40,11.41,63.00,,
2,1507390,Dikson,73.51,80.55,-5.84,95,100,5.39,-5.84,,
3,2075265,Busselton,-33.65,115.33,90.73,24,97,5.23,93.00,,
4,4030556,Rikitea,-23.12,-134.97,79.11,77,0,10.22,79.11,,


In [8]:

params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key
    }
    
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
for index, row in hotel_df.iterrows():

    # get location from df
    target_location = (f"{row['lat']},{row['lng']}")
    target_city = row['city']

    # add location to params dict
    params['location'] = target_location
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {target_city}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
                       
                       
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['plus_code']['compound_code']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Mataura.
Closest hotel is Falls Hotel.
------------
Retrieving Results for Index 1: Hong Kong.
Closest hotel is Four Seasons Hotel Hong Kong.
------------
Retrieving Results for Index 2: Dikson.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: Busselton.
Closest hotel is Busselton Beach House.
------------
Retrieving Results for Index 4: Rikitea.
Closest hotel is Pension Maro'i.
------------
Retrieving Results for Index 5: Hofn.
Closest hotel is Sólgerði.
------------
Retrieving Results for Index 6: Karratha.
Closest hotel is ibis Styles Karratha.
------------
Retrieving Results for Index 7: Kapaa.
Closest hotel is Honu'ea International Hostel Kauai.
------------
Retrieving Results for Index 8: Shenkursk.
Closest hotel is "Vaga" hotel.
------------
Retrieving Results for Index 9: Bethel.
Closest hotel is Ibrahim Haddad.
------------
Retrieving Results for Index 10: Agirish.
Missing field/result... skipping.
------------
Retr

Closest hotel is Morgan City Motel & Apartments.
------------
Retrieving Results for Index 89: Isla Mujeres.
Closest hotel is Casa Yaxk’in.
------------
Retrieving Results for Index 90: Kodiak.
Closest hotel is Kodiak Compass Suites.
------------
Retrieving Results for Index 91: Ugoofaaru.
Closest hotel is Orchid.
------------
Retrieving Results for Index 92: Saint George.
Closest hotel is Jubilee Home.
------------
Retrieving Results for Index 93: Kieta.
Closest hotel is Arawa Travellers Inn.
------------
Retrieving Results for Index 94: Port-Gentil.
Closest hotel is Hotel Tara-Mé.
------------
Retrieving Results for Index 95: Flinders.
Closest hotel is Shellharbour Village Escape.
------------
Retrieving Results for Index 96: Guarda.
Closest hotel is Casa da Sé / Guesthouse da Sé.
------------
Retrieving Results for Index 97: Upernavik.
Closest hotel is Gina's Guesthouse - Upernavik.
------------
Retrieving Results for Index 98: Buraidah.
Closest hotel is فندق الديره.
------------
Re

Closest hotel is German Patio Bar.
------------
Retrieving Results for Index 176: Hambantota.
Closest hotel is Happy Beach Hotel.
------------
Retrieving Results for Index 177: Kolokani.
Closest hotel is Lycee Famolo Coulibalibali De Kolokani.
------------
Retrieving Results for Index 178: Toamasina.
Closest hotel is Villa EMJIACHE.
------------
Retrieving Results for Index 179: Balkhash.
Closest hotel is Kazakhstan.
------------
Retrieving Results for Index 180: Dingle.
Closest hotel is Camp Pasica.
------------
Retrieving Results for Index 181: Am Timan.
Missing field/result... skipping.
------------
Retrieving Results for Index 182: Talcahuano.
Closest hotel is Elías del Carmen Gajardo Tapia.
Missing field/result... skipping.
------------
Retrieving Results for Index 183: Ixtapa.
Closest hotel is La Casa de SAM.
------------
Retrieving Results for Index 184: Hobart.
Closest hotel is Mantra on Collins Hobart.
------------
Retrieving Results for Index 185: Mayor Pablo Lagerenza.
Missi

Closest hotel is Frankfaus.
------------
Retrieving Results for Index 261: Capao da Canoa.
Closest hotel is Hotel Imperial F Sheffer.
------------
Retrieving Results for Index 262: Montepuez.
Closest hotel is RESIDÊNCIAL LANCHONETE.
------------
Retrieving Results for Index 263: Port Blair.
Closest hotel is SRI NITHI TRAVELS (Andaman).
------------
Retrieving Results for Index 264: Lyubech.
Missing field/result... skipping.
------------
Retrieving Results for Index 265: Mount Gambier.
Closest hotel is Adela Cottage.
------------
Retrieving Results for Index 266: Lakes Entrance.
Closest hotel is Lakeside Motel Waterfront.
------------
Retrieving Results for Index 267: San Quintin.
Closest hotel is Hotel Jardines Baja.
------------
Retrieving Results for Index 268: Skelleftea.
Closest hotel is Quality Hotel Skellefteå Stadshotell.
------------
Retrieving Results for Index 269: Coyhaique.
Closest hotel is HUEMUL PATAGÓN.
------------
Retrieving Results for Index 270: Pacifica.
Closest hot

Closest hotel is Motel On York.
------------
Retrieving Results for Index 348: Ambon.
Closest hotel is Penginapa Violet.
------------
Retrieving Results for Index 349: Departamento de Maldonado.
Closest hotel is La Cimarrona.
------------
Retrieving Results for Index 350: Pevek.
Closest hotel is Severnoye Zoloto.
------------
Retrieving Results for Index 351: San Vicente.
Closest hotel is Hospedaje La Luz.
------------
Retrieving Results for Index 352: Guia de Isora.
Closest hotel is Francisco Mesa.
------------
Retrieving Results for Index 353: Brae.
Closest hotel is Moorfield Hotel.
------------
Retrieving Results for Index 354: Isabela.
Closest hotel is Hotel Sofia.
------------
Retrieving Results for Index 355: Haizhou.
Closest hotel is Traffic Hotel.
------------
Retrieving Results for Index 356: Dandong.
Closest hotel is Erqing Hostel.
------------
Retrieving Results for Index 357: Tulun.
Closest hotel is Centralnaya Hotel.
------------
Retrieving Results for Index 358: Plouzane.

Closest hotel is Al Sahwa New Building.
------------
Retrieving Results for Index 434: Mujiayingzi.
Closest hotel is Taicheng Hotel.
------------
Retrieving Results for Index 435: Bentiu.
Closest hotel is Mandela hôtel.
------------
Retrieving Results for Index 436: Lamu.
Closest hotel is Jannatan Hotel.
------------
Retrieving Results for Index 437: Broken Hill.
Closest hotel is Excelsior Hotel.
Missing field/result... skipping.
------------
Retrieving Results for Index 438: Zhanakorgan.
Closest hotel is "Ақпалас" қонақ үйі.
------------
Retrieving Results for Index 439: Jinchang.
Closest hotel is 龙首山庄.
------------
Retrieving Results for Index 440: Robinsonville.
Closest hotel is Townhome.
------------
Retrieving Results for Index 441: Laconia.
Closest hotel is Pleasant View Park -Management.
------------
Retrieving Results for Index 442: Port Shepstone.
Closest hotel is The Holiday Club.
------------
Retrieving Results for Index 443: Sherghati.
Closest hotel is Upkar Hotel.
--------

Closest hotel is Merwede Guest House.
------------
Retrieving Results for Index 522: Gaza.
Closest hotel is منزل أبناء المرحوم الشيخ حسن لبد.
Missing field/result... skipping.
------------
Retrieving Results for Index 523: Bandarban.
Closest hotel is Hotel Sangu.
------------
Retrieving Results for Index 524: Acari.
Closest hotel is Hospedaje Ximens.
------------
Retrieving Results for Index 525: Anzio.
Closest hotel is B&B Villa Claudia.
------------
Retrieving Results for Index 526: Monte Patria.
Closest hotel is Hostal korona.
------------
Retrieving Results for Index 527: Ola.
Missing field/result... skipping.
------------
Retrieving Results for Index 528: Bougouni.
Closest hotel is Non Stop.
------------
Retrieving Results for Index 529: Ongandjera.
Closest hotel is Onesto Guesthouse.
------------
Retrieving Results for Index 530: Fredonia.
Closest hotel is Edwards Waterhouse Inn.
------------
Retrieving Results for Index 531: Bay Roberts.
Closest hotel is Klondyke Hotel.
--------

In [9]:
hotel_df.head(3)

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp,Hotel Name,address
0,6201424,Mataura,-46.19,168.86,77.65,58,64,0.58,77.65,Falls Hotel,"RV47+HX Mataura, Southland, New Zealand"
1,1819729,Hong Kong,22.29,114.16,61.66,77,40,11.41,63.00,Four Seasons Hotel Hong Kong,"75P4+HQ Central, Hong Kong"
2,1507390,Dikson,73.51,80.55,-5.84,95,100,5.39,-5.84,,


In [10]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, 
                                  fill_color='rgba(0, 150, 0, 0.4)',
                                  stroke_color='rgba(0, 0, 150, 0.4)',
                                  scale=2,
                                 info_box_content = hotel_info)
fig.add_layer(hotel_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))